In [1]:
!git clone https://github.com/Zeleni9/pytorch-wgan.git

Cloning into 'pytorch-wgan'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 68 (delta 14), reused 16 (delta 7), pack-reused 28
Unpacking objects: 100% (68/68), done.


In [2]:
!mkdir data_faces && wget https://s3-us-west-1.amazonaws.com/udacity-dlnfd/datasets/celeba.zip 

--2020-11-03 00:39:29--  https://s3-us-west-1.amazonaws.com/udacity-dlnfd/datasets/celeba.zip
Resolving s3-us-west-1.amazonaws.com (s3-us-west-1.amazonaws.com)... 52.219.121.0
Connecting to s3-us-west-1.amazonaws.com (s3-us-west-1.amazonaws.com)|52.219.121.0|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1443490838 (1.3G) [application/zip]
Saving to: ‘celeba.zip’

celeba.zip          100%[===================>]   1.34G  19.7MB/s    in 1m 40s  

2020-11-03 00:41:09 (13.7 MB/s) - ‘celeba.zip’ saved [1443490838/1443490838]



In [1]:
import zipfile

with zipfile.ZipFile("celeba.zip","r") as zip_ref:
  zip_ref.extractall("data_faces/")

In [2]:
import os
root = 'data_faces/img_align_celeba'
img_list = os.listdir(root)
print(len(img_list))

202599


In [1]:
!pip install Pillow==4.0.0
!pip install PIL
!pip install image
import PIL.Image as Image
import torch
import torchvision
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import DataLoader
""" data """
crop_size = 108
re_size = 32
offset_height = (218 - crop_size) // 2
offset_width = (178 - crop_size) // 2
crop = lambda x: x[:, offset_height:offset_height + crop_size, offset_width:offset_width + crop_size]

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Lambda(crop),
     transforms.ToPILImage(),
     transforms.Scale(size=(re_size, re_size), interpolation=Image.BICUBIC),
     transforms.ToTensor(),
     transforms.Normalize(mean=[0.5] * 3, std=[0.5] * 3)])

batch_size = 64
celeba_data = datasets.ImageFolder('./data_faces', transform=transform)
data_loader = DataLoader(celeba_data,batch_size=batch_size,shuffle=True)

ERROR: Could not find a version that satisfies the requirement PIL (from versions: none)
ERROR: No matching distribution found for PIL


/opt/conda/lib/python3.7/site-packages/torchvision/transforms/transforms.py:219: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  warnings.warn("The use of the transforms.Scale transform is deprecated, " +


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch import autograd
import time as t
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import os

from itertools import chain
from torchvision import utils

SAVE_PER_TIMES = 100

In [3]:
class Generator(torch.nn.Module):
    def __init__(self, channels):
        super().__init__()
        # Filters [1024, 512, 256]
        # Input_dim = 100
        # Output_dim = C (number of channels)
        self.main_module = nn.Sequential(
            # Z latent vector 100
            nn.ConvTranspose2d(in_channels=100, out_channels=1024, kernel_size=4, stride=1, padding=0),
            nn.BatchNorm2d(num_features=1024),
            nn.ReLU(True),

            # State (1024x4x4)
            nn.ConvTranspose2d(in_channels=1024, out_channels=512, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(num_features=512),
            nn.ReLU(True),

            # State (512x8x8)
            nn.ConvTranspose2d(in_channels=512, out_channels=256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(num_features=256),
            nn.ReLU(True),

            # State (256x16x16)
            nn.ConvTranspose2d(in_channels=256, out_channels=channels, kernel_size=4, stride=2, padding=1))
            # output of main module --> Image (Cx32x32)

        self.output = nn.Tanh()

    def forward(self, x):
        x = self.main_module(x)
        return self.output(x)


In [4]:
class Discriminator(torch.nn.Module):
    def __init__(self, channels):
        super().__init__()
        # Filters [256, 512, 1024]
        # Input_dim = channels (Cx64x64)
        # Output_dim = 1
        self.main_module = nn.Sequential(
            # Omitting batch normalization in critic because our new penalized training objective (WGAN with gradient penalty) is no longer valid
            # in this setting, since we penalize the norm of the critic's gradient with respect to each input independently and not the enitre batch.
            # There is not good & fast implementation of layer normalization --> using per instance normalization nn.InstanceNorm2d()
            # Image (Cx32x32)
            nn.Conv2d(in_channels=channels, out_channels=256, kernel_size=4, stride=2, padding=1),
            nn.InstanceNorm2d(256, affine=True),
            nn.LeakyReLU(0.2, inplace=True),

            # State (256x16x16)
            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=4, stride=2, padding=1),
            nn.InstanceNorm2d(512, affine=True),
            nn.LeakyReLU(0.2, inplace=True),

            # State (512x8x8)
            nn.Conv2d(in_channels=512, out_channels=1024, kernel_size=4, stride=2, padding=1),
            nn.InstanceNorm2d(1024, affine=True),
            nn.LeakyReLU(0.2, inplace=True))
            # output of main module --> State (1024x4x4)

        self.output = nn.Sequential(
            # The output of D is no longer a probability, we do not apply sigmoid at the output of D.
            nn.Conv2d(in_channels=1024, out_channels=1, kernel_size=4, stride=1, padding=0))


    def forward(self, x):
        x = self.main_module(x)
        return self.output(x)

In [5]:
class WGAN_GP(object):
    def __init__(self):
        print("WGAN_GradientPenalty init model.")
        self.G = Generator(3)
        self.D = Discriminator(3)
        self.C = 3

        # Check if cuda is available
        cuda = True if torch.cuda.is_available() else False
        self.check_cuda(cuda)

        # WGAN values from paper
        self.learning_rate = 1e-4
        self.b1 = 0.5
        self.b2 = 0.999
        self.batch_size = 64

        # WGAN_gradient penalty uses ADAM
        self.d_optimizer = optim.Adam(self.D.parameters(), lr=self.learning_rate, betas=(self.b1, self.b2))
        self.g_optimizer = optim.Adam(self.G.parameters(), lr=self.learning_rate, betas=(self.b1, self.b2))

        self.number_of_images = 10

        self.critic_iter = 5
        self.lambda_term = 10

    def get_torch_variable(self, arg):
        if self.cuda:
            return Variable(arg).cuda(self.cuda_index)
        else:
            return Variable(arg)

    def check_cuda(self, cuda_flag=False):
        print(cuda_flag)
        if cuda_flag:
            self.cuda_index = 0
            self.cuda = True
            self.D.cuda(self.cuda_index)
            self.G.cuda(self.cuda_index)
            print("Cuda enabled flag: {}".format(self.cuda))
        else:
            self.cuda = False


    def train(self, train_loader):
        self.t_begin = t.time()
        self.file = open("inception_score_graph.txt", "w")

        # Now batches are callable self.data.next()
        self.data = self.get_infinite_batches(train_loader)

        one = torch.tensor(1, dtype=torch.float)
        mone = one * -1
        if self.cuda:
            one = one.cuda(self.cuda_index)
            mone = mone.cuda(self.cuda_index)

        for g_iter in range(100000):
            # Requires grad, Generator requires_grad = False
            for p in self.D.parameters():
                p.requires_grad = True

            d_loss_real = 0
            d_loss_fake = 0
            Wasserstein_D = 0
            # Train Dicriminator forward-loss-backward-update self.critic_iter times while 1 Generator forward-loss-backward-update
            for d_iter in range(self.critic_iter):
                self.D.zero_grad()

                images = self.data.__next__()
                # Check for batch to have full batch_size
                if (images.size()[0] != self.batch_size):
                    continue

                z = torch.rand((self.batch_size, 100, 1, 1))

                images, z = self.get_torch_variable(images), self.get_torch_variable(z)

                # Train discriminator
                # WGAN - Training discriminator more iterations than generator
                # Train with real images
                d_loss_real = self.D(images)
                d_loss_real = d_loss_real.mean()
                d_loss_real.backward(mone)

                # Train with fake images
                z = self.get_torch_variable(torch.randn(self.batch_size, 100, 1, 1))

                fake_images = self.G(z)
                d_loss_fake = self.D(fake_images)
                d_loss_fake = d_loss_fake.mean()
                d_loss_fake.backward(one)

                # Train with gradient penalty
                gradient_penalty = self.calculate_gradient_penalty(images.data, fake_images.data)
                gradient_penalty.backward()


                d_loss = d_loss_fake - d_loss_real + gradient_penalty
                Wasserstein_D = d_loss_real - d_loss_fake
                self.d_optimizer.step()
                print(f'  Discriminator iteration: {d_iter}/{self.critic_iter}, loss_fake: {d_loss_fake}, loss_real: {d_loss_real}')

            # Generator update
            for p in self.D.parameters():
                p.requires_grad = False  # to avoid computation

            self.G.zero_grad()
            # train generator
            # compute loss with fake images
            z = self.get_torch_variable(torch.randn(self.batch_size, 100, 1, 1))
            fake_images = self.G(z)
            g_loss = self.D(fake_images)
            g_loss = g_loss.mean()
            g_loss.backward(mone)
            g_cost = -g_loss
            self.g_optimizer.step()
            
            # Saving model and sampling images every 1000th generator iterations
            if (g_iter) % SAVE_PER_TIMES == 0:
                self.save_model()

                if not os.path.exists('training_result_images/'):
                    os.makedirs('training_result_images/')

                # Denormalize images and save them in grid 8x8
                z = self.get_torch_variable(torch.randn(800, 100, 1, 1))
                samples = self.G(z)
                samples = samples.mul(0.5).add(0.5)
                samples = samples.data.cpu()[:64]
                grid = utils.make_grid(samples)
                utils.save_image(grid, 'training_result_images/img_generatori_iter_{}.png'.format(str(g_iter).zfill(3)))

                # Testing
                time = t.time() - self.t_begin
                print("Generator iter: {}".format(g_iter))
                print("Time {}".format(time))


        self.t_end = t.time()
        print('Time of training-{}'.format((self.t_end - self.t_begin)))

        # Save the trained parameters
        self.save_model()

    def calculate_gradient_penalty(self, real_images, fake_images):
        eta = torch.FloatTensor(self.batch_size,1,1,1).uniform_(0,1)
        eta = eta.expand(self.batch_size, real_images.size(1), real_images.size(2), real_images.size(3))
        if self.cuda:
            eta = eta.cuda(self.cuda_index)
        else:
            eta = eta
        interpolated = eta * real_images + ((1 - eta) * fake_images)

        if self.cuda:
            interpolated = interpolated.cuda(self.cuda_index)
        else:
            interpolated = interpolated

        # define it to calculate gradient
        interpolated = Variable(interpolated, requires_grad=True)

        # calculate probability of interpolated examples
        prob_interpolated = self.D(interpolated)

        # calculate gradients of probabilities with respect to examples
        gradients = autograd.grad(outputs=prob_interpolated, inputs=interpolated,
                               grad_outputs=torch.ones(
                                   prob_interpolated.size()).cuda(self.cuda_index) if self.cuda else torch.ones(
                                   prob_interpolated.size()),
                               create_graph=True, retain_graph=True)[0]

        grad_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean() * self.lambda_term
        return grad_penalty

    def save_model(self):
        torch.save(self.G.state_dict(), './generator.pkl')
        torch.save(self.D.state_dict(), './discriminator.pkl')
        print('Models save to ./generator.pkl & ./discriminator.pkl ')

    def get_infinite_batches(self, data_loader):
        while True:
            for i, (images, _) in enumerate(data_loader):
                yield images

In [ ]:
model = WGAN_GP()
model.train(data_loader)

WGAN_GradientPenalty init model.
True
Cuda enabled flag: True
  Discriminator iteration: 0/5, loss_fake: -0.2551889717578888, loss_real: -0.24922379851341248
  Discriminator iteration: 1/5, loss_fake: -2.9142959117889404, loss_real: 5.711061477661133
  Discriminator iteration: 2/5, loss_fake: -5.750449180603027, loss_real: 11.772893905639648
  Discriminator iteration: 3/5, loss_fake: -8.707605361938477, loss_real: 15.103328704833984
  Discriminator iteration: 4/5, loss_fake: -11.978415489196777, loss_real: 19.259510040283203
Models save to ./generator.pkl & ./discriminator.pkl 
Generator iter: 0
Time 7.820501327514648
  Discriminator iteration: 0/5, loss_fake: 5.972305774688721, loss_real: 20.658313751220703
  Discriminator iteration: 1/5, loss_fake: 1.1600805521011353, loss_real: 23.673343658447266
  Discriminator iteration: 2/5, loss_fake: -5.830830097198486, loss_real: 26.738039016723633
  Discriminator iteration: 3/5, loss_fake: -13.024316787719727, loss_real: 26.769460678100586
  

  Discriminator iteration: 3/5, loss_fake: -117.03427124023438, loss_real: 127.14827728271484
  Discriminator iteration: 4/5, loss_fake: -118.79510498046875, loss_real: 128.87774658203125
  Discriminator iteration: 0/5, loss_fake: -101.59878540039062, loss_real: 128.9034423828125
  Discriminator iteration: 1/5, loss_fake: -116.67727661132812, loss_real: 130.67788696289062
  Discriminator iteration: 2/5, loss_fake: -119.84063720703125, loss_real: 130.63949584960938
  Discriminator iteration: 3/5, loss_fake: -122.25221252441406, loss_real: 130.5605926513672
  Discriminator iteration: 4/5, loss_fake: -124.16439056396484, loss_real: 132.4105224609375
  Discriminator iteration: 0/5, loss_fake: -120.57453155517578, loss_real: 134.38279724121094
  Discriminator iteration: 1/5, loss_fake: -123.82160949707031, loss_real: 135.3157196044922
  Discriminator iteration: 2/5, loss_fake: -126.22583770751953, loss_real: 135.4681396484375
  Discriminator iteration: 3/5, loss_fake: -128.26376342773438, l

  Discriminator iteration: 2/5, loss_fake: -215.71728515625, loss_real: 226.2406005859375
  Discriminator iteration: 3/5, loss_fake: -218.25213623046875, loss_real: 227.47613525390625
  Discriminator iteration: 4/5, loss_fake: -219.96536254882812, loss_real: 229.226806640625
  Discriminator iteration: 0/5, loss_fake: -209.7218017578125, loss_real: 231.65274047851562
  Discriminator iteration: 1/5, loss_fake: -217.56857299804688, loss_real: 232.4617156982422
  Discriminator iteration: 2/5, loss_fake: -221.34585571289062, loss_real: 231.66952514648438
  Discriminator iteration: 3/5, loss_fake: -223.61172485351562, loss_real: 231.10791015625
  Discriminator iteration: 4/5, loss_fake: -225.55027770996094, loss_real: 234.4605255126953
  Discriminator iteration: 0/5, loss_fake: -216.8622589111328, loss_real: 235.65359497070312
  Discriminator iteration: 1/5, loss_fake: -223.50363159179688, loss_real: 236.82278442382812
  Discriminator iteration: 2/5, loss_fake: -227.12359619140625, loss_real

  Discriminator iteration: 1/5, loss_fake: -308.62969970703125, loss_real: 315.90850830078125
  Discriminator iteration: 2/5, loss_fake: -312.49969482421875, loss_real: 307.7266845703125
  Discriminator iteration: 3/5, loss_fake: -312.6156005859375, loss_real: 324.84698486328125
  Discriminator iteration: 4/5, loss_fake: -315.2604064941406, loss_real: 328.3008728027344
  Discriminator iteration: 0/5, loss_fake: -287.55279541015625, loss_real: 330.00579833984375
  Discriminator iteration: 1/5, loss_fake: -310.5948181152344, loss_real: 323.31298828125
  Discriminator iteration: 2/5, loss_fake: -317.069091796875, loss_real: 317.4159240722656
  Discriminator iteration: 3/5, loss_fake: -320.2555236816406, loss_real: 331.8172302246094
  Discriminator iteration: 4/5, loss_fake: -320.5269775390625, loss_real: 331.762451171875
  Discriminator iteration: 0/5, loss_fake: -291.40740966796875, loss_real: 332.7158508300781
  Discriminator iteration: 1/5, loss_fake: -317.7861328125, loss_real: 325.96

  Discriminator iteration: 1/5, loss_fake: -404.0307312011719, loss_real: 421.27679443359375
  Discriminator iteration: 2/5, loss_fake: -410.2391052246094, loss_real: 425.3466796875
  Discriminator iteration: 3/5, loss_fake: -408.6390686035156, loss_real: 427.1651916503906
  Discriminator iteration: 4/5, loss_fake: -412.7672119140625, loss_real: 427.99554443359375
  Discriminator iteration: 0/5, loss_fake: -403.04791259765625, loss_real: 430.40057373046875
  Discriminator iteration: 1/5, loss_fake: -410.55157470703125, loss_real: 425.55029296875
  Discriminator iteration: 2/5, loss_fake: -416.7367858886719, loss_real: 432.3984375
  Discriminator iteration: 3/5, loss_fake: -415.3363037109375, loss_real: 434.64361572265625
  Discriminator iteration: 4/5, loss_fake: -414.102294921875, loss_real: 436.78106689453125
  Discriminator iteration: 0/5, loss_fake: -404.7166442871094, loss_real: 432.7082824707031
  Discriminator iteration: 1/5, loss_fake: -419.2371826171875, loss_real: 436.7028198

...

  Discriminator iteration: 0/5, loss_fake: -1320.577392578125, loss_real: -1063.73779296875
  Discriminator iteration: 1/5, loss_fake: -1297.7799072265625, loss_real: -1005.8500366210938
  Discriminator iteration: 2/5, loss_fake: -1304.75634765625, loss_real: -997.431884765625
  Discriminator iteration: 3/5, loss_fake: -1339.38623046875, loss_real: -1056.4971923828125
  Discriminator iteration: 4/5, loss_fake: -1313.3275146484375, loss_real: -1039.772216796875
  Discriminator iteration: 0/5, loss_fake: -1341.677490234375, loss_real: -1134.85107421875
  Discriminator iteration: 1/5, loss_fake: -1384.828857421875, loss_real: -1114.7939453125
  Discriminator iteration: 2/5, loss_fake: -1312.145263671875, loss_real: -1042.6422119140625
  Discriminator iteration: 3/5, loss_fake: -1307.59375, loss_real: -967.2955322265625
  Discriminator iteration: 4/5, loss_fake: -1263.2802734375, loss_real: -911.508544921875
  Discriminator iteration: 0/5, loss_fake: -1283.261962890625, loss_real: -956.038

  Discriminator iteration: 0/5, loss_fake: -1166.593505859375, loss_real: -941.310546875
  Discriminator iteration: 1/5, loss_fake: -1261.8291015625, loss_real: -1063.798583984375
  Discriminator iteration: 2/5, loss_fake: -1373.2205810546875, loss_real: -1105.8358154296875
  Discriminator iteration: 3/5, loss_fake: -1347.1318359375, loss_real: -1096.9801025390625
  Discriminator iteration: 4/5, loss_fake: -1349.6181640625, loss_real: -1030.3115234375
  Discriminator iteration: 0/5, loss_fake: -1220.215576171875, loss_real: -950.7423706054688
  Discriminator iteration: 1/5, loss_fake: -1294.01611328125, loss_real: -977.074462890625
  Discriminator iteration: 2/5, loss_fake: -1317.121337890625, loss_real: -933.92724609375
  Discriminator iteration: 3/5, loss_fake: -1350.26025390625, loss_real: -1032.232666015625
  Discriminator iteration: 4/5, loss_fake: -1371.1488037109375, loss_real: -941.746337890625
  Discriminator iteration: 0/5, loss_fake: -1309.50390625, loss_real: -964.372436523

  Discriminator iteration: 4/5, loss_fake: -1371.822021484375, loss_real: -1071.654052734375
  Discriminator iteration: 0/5, loss_fake: -1233.825439453125, loss_real: -986.7799072265625
  Discriminator iteration: 1/5, loss_fake: -1262.29833984375, loss_real: -993.777587890625
  Discriminator iteration: 2/5, loss_fake: -1363.13232421875, loss_real: -1093.0950927734375
  Discriminator iteration: 3/5, loss_fake: -1396.131103515625, loss_real: -1089.924560546875
  Discriminator iteration: 4/5, loss_fake: -1392.20458984375, loss_real: -1113.999267578125
  Discriminator iteration: 0/5, loss_fake: -1311.023681640625, loss_real: -971.7445678710938
  Discriminator iteration: 1/5, loss_fake: -1380.5648193359375, loss_real: -1051.8983154296875
  Discriminator iteration: 2/5, loss_fake: -1340.07666015625, loss_real: -956.8236083984375
  Discriminator iteration: 3/5, loss_fake: -1330.072509765625, loss_real: -967.4708862304688
  Discriminator iteration: 4/5, loss_fake: -1325.9979248046875, loss_rea

  Discriminator iteration: 2/5, loss_fake: -1297.813720703125, loss_real: -1015.477294921875
  Discriminator iteration: 3/5, loss_fake: -1316.12890625, loss_real: -990.85400390625
  Discriminator iteration: 4/5, loss_fake: -1286.03125, loss_real: -970.1665649414062
  Discriminator iteration: 0/5, loss_fake: -1185.3211669921875, loss_real: -789.52783203125
  Discriminator iteration: 1/5, loss_fake: -1265.88232421875, loss_real: -1002.9339599609375
  Discriminator iteration: 2/5, loss_fake: -1346.382568359375, loss_real: -1042.0166015625
  Discriminator iteration: 3/5, loss_fake: -1328.3759765625, loss_real: -1092.275390625
  Discriminator iteration: 4/5, loss_fake: -1361.25732421875, loss_real: -1024.1090087890625
  Discriminator iteration: 0/5, loss_fake: -1312.9893798828125, loss_real: -1036.345703125
  Discriminator iteration: 1/5, loss_fake: -1312.4417724609375, loss_real: -987.79541015625
  Discriminator iteration: 2/5, loss_fake: -1376.0106201171875, loss_real: -1097.9609375
  Dis

  Discriminator iteration: 2/5, loss_fake: -1395.7542724609375, loss_real: -1092.2886962890625
  Discriminator iteration: 3/5, loss_fake: -1385.080810546875, loss_real: -1076.439208984375
  Discriminator iteration: 4/5, loss_fake: -1316.6014404296875, loss_real: -1016.546875
  Discriminator iteration: 0/5, loss_fake: -1318.2840576171875, loss_real: -996.5319213867188
  Discriminator iteration: 1/5, loss_fake: -1321.598876953125, loss_real: -1069.490478515625
  Discriminator iteration: 2/5, loss_fake: -1334.68896484375, loss_real: -1035.855712890625
  Discriminator iteration: 3/5, loss_fake: -1371.9737548828125, loss_real: -1087.1749267578125
  Discriminator iteration: 4/5, loss_fake: -1349.3974609375, loss_real: -992.7369384765625
  Discriminator iteration: 0/5, loss_fake: -1289.145751953125, loss_real: -1052.856201171875
  Discriminator iteration: 1/5, loss_fake: -1344.852783203125, loss_real: -1019.3471069335938
  Discriminator iteration: 2/5, loss_fake: -1317.421630859375, loss_real

In [6]:
# Loading the model
model = WGAN_GP()
model.G.load_state_dict(torch.load('./latesh_epoch_generator.pkl'))
model.D.load_state_dict(torch.load('./latesh_epoch_discriminator.pkl'))

WGAN_GradientPenalty init model.
True
Cuda enabled flag: True


<All keys matched successfully>

In [8]:
# Generating Samples Images
import numpy
%matplotlib inline

for g_iter in range(100):
    z = model.get_torch_variable(torch.randn(64, 100, 1, 1))
    samples = model.G(z)
    samples = samples.mul(0.5).add(0.5)
    samples = samples.data.cpu()[:64]
    print(samples[0].shape)
    [utils.save_image(samples[sample_i], 'sep_images_folder/img_generatori_iter_{}-{}.png'.format(str(g_iter).zfill(3), sample_i)) for sample_i in range(64)]

torch.Size([3, 32, 32])
torch.Size([3, 32, 32])
torch.Size([3, 32, 32])
torch.Size([3, 32, 32])
torch.Size([3, 32, 32])
torch.Size([3, 32, 32])
torch.Size([3, 32, 32])
torch.Size([3, 32, 32])
torch.Size([3, 32, 32])
torch.Size([3, 32, 32])
torch.Size([3, 32, 32])
torch.Size([3, 32, 32])
torch.Size([3, 32, 32])
torch.Size([3, 32, 32])
torch.Size([3, 32, 32])
torch.Size([3, 32, 32])
torch.Size([3, 32, 32])
torch.Size([3, 32, 32])
torch.Size([3, 32, 32])
torch.Size([3, 32, 32])
torch.Size([3, 32, 32])
torch.Size([3, 32, 32])
torch.Size([3, 32, 32])
torch.Size([3, 32, 32])
torch.Size([3, 32, 32])
torch.Size([3, 32, 32])
torch.Size([3, 32, 32])
torch.Size([3, 32, 32])
torch.Size([3, 32, 32])
torch.Size([3, 32, 32])
torch.Size([3, 32, 32])
torch.Size([3, 32, 32])
torch.Size([3, 32, 32])
torch.Size([3, 32, 32])
torch.Size([3, 32, 32])
torch.Size([3, 32, 32])
torch.Size([3, 32, 32])
torch.Size([3, 32, 32])
torch.Size([3, 32, 32])
torch.Size([3, 32, 32])
torch.Size([3, 32, 32])
torch.Size([3, 3